In [2]:
import numpy as np

In [3]:
#Подготовка входных данных
print("="*60)
print("1. ПОДГОТОВКА ВХОДНЫХ ДАННЫХ ДЛЯ МЕТОДА ПРОГОНКИ")
print("="*60)
n = 10

#Создаем трехдиагональную матрицу, удовлетворяющую условиям теоремы
print("\nа) Создание трехдиагональной матрицы A размера", n, "×", n)
a = np.zeros(n)  # нижняя диагональ (a[0] не используется, обычно a[0] = 0)
b = np.zeros(n)  # главная диагональ
c = np.zeros(n)  # верхняя диагональ (c[n-1] не используется, обычно c[n-1] = 0)
np.random.seed(42)

b[0] = 4.0
c[0] = 1.0
a[0] = 0.0

a[n-1] = 2.0
b[n-1] = 5.0
c[n-1] = 0.0

for i in range(1, n-1):
    a[i] = np.random.uniform(0.5, 1.5)
    c[i] = np.random.uniform(0.5, 1.5)
    b[i] = abs(a[i]) + abs(c[i]) + np.random.uniform(1.0, 2.0)
d = np.random.uniform(1, 10, n)
print("\nДиагонали матрицы A:")
print("Нижняя диагональ a:", np.round(a, 3))
print("Главная диагональ b:", np.round(b, 3))
print("Верхняя диагональ c:", np.round(c, 3))
print("\nВектор правой части d:", np.round(d, 3))

#Проверяем условия применимости метода прогонки
print("\nб) Проверка условий применимости метода прогонки:")
conditions_met = True
for k in range(n):
    cond1 = abs(b[k]) >= abs(a[k]) + abs(c[k])
    cond2 = abs(b[k]) > abs(a[k])
    if not (cond1 and cond2):
        conditions_met = False
        print(f"  Условия не выполнены для k={k+1}: |b|={abs(b[k]):.3f}, |a|+|c|={abs(a[k])+abs(c[k]):.3f}")
if conditions_met:
    print("  ✓ Все условия теоремы 1 выполнены:")
    print("    |b_k| ≥ |a_k| + |c_k| для всех k")
    print("    |b_k| > |a_k| для всех k")
    print("  → Метод прогонки гарантированно применим")
else:
    print("  ✗ Некоторые условия теоремы не выполнены")
    print("  → Метод прогонки может быть неустойчив")

1. ПОДГОТОВКА ВХОДНЫХ ДАННЫХ ДЛЯ МЕТОДА ПРОГОНКИ

а) Создание трехдиагональной матрицы A размера 10 × 10

Диагонали матрицы A:
Нижняя диагональ a: [0.    0.875 1.099 0.558 1.208 1.332 0.683 0.932 0.639 2.   ]
Главная диагональ b: [4.    4.057 2.911 3.525 3.699 3.227 3.012 3.335 2.798 5.   ]
Верхняя диагональ c: [1.    1.451 0.656 1.366 0.521 0.712 0.804 0.791 0.792 0.   ]

Вектор правой части d: [5.105 8.067 2.797 5.628 6.332 1.418 6.468 2.535 1.585 9.54 ]

б) Проверка условий применимости метода прогонки:
  ✓ Все условия теоремы 1 выполнены:
    |b_k| ≥ |a_k| + |c_k| для всех k
    |b_k| > |a_k| для всех k
  → Метод прогонки гарантированно применим


In [4]:
#Метод томаса
print("\n" + "="*60)
print("2. РЕАЛИЗАЦИЯ МЕТОДА ПРОГОНКИ (АЛГОРИТМ ТОМАСА)")
print("="*60)

def thomas_algorithm(a, b, c, d):
    n = len(d)
    alpha = np.zeros(n-1)
    gamma = np.zeros(n)
    beta = np.zeros(n)
    x = np.zeros(n)
    gamma[0] = b[0]
    beta[0] = d[0] / gamma[0]
    if n > 1:
        alpha[0] = -c[0] / gamma[0]
    for i in range(1, n-1):
        gamma[i] = b[i] + a[i] * alpha[i-1]
        beta[i] = (d[i] - a[i] * beta[i-1]) / gamma[i]
        alpha[i] = -c[i] / gamma[i]
    if n > 1:
        i = n-1
        gamma[i] = b[i] + a[i] * alpha[i-1]
        beta[i] = (d[i] - a[i] * beta[i-1]) / gamma[i]
    else:
        beta[0] = d[0] / b[0]
    x[n-1] = beta[n-1]
    for i in range(n-2, -1, -1):
        x[i] = alpha[i] * x[i+1] + beta[i]
    return x, alpha, gamma

def check_solution(a, b, c, d, x):
    n = len(d)
    Ax = np.zeros(n)
    Ax[0] = b[0] * x[0] + c[0] * x[1] if n > 1 else b[0] * x[0]
    for i in range(1, n-1):
        Ax[i] = a[i] * x[i-1] + b[i] * x[i] + c[i] * x[i+1]
    if n > 1:
        Ax[n-1] = a[n-1] * x[n-2] + b[n-1] * x[n-1]
    residual = Ax - d
    norm = np.linalg.norm(residual)
    return norm, residual

print("\nРешение системы методом прогонки...")
x_solution, alpha_coeffs, gamma_vals = thomas_algorithm(a, b, c, d)

print(f"\nРешение x:")
for i, val in enumerate(x_solution):
    print(f"  x[{i+1}] = {val:.6f}")

residual_norm, residual_vector = check_solution(a, b, c, d, x_solution)
print(f"\nПроверка решения:")
print(f"  Норма невязки ||Ax - d|| = {residual_norm:.2e}")
print(f"  Максимальная невязка = {np.max(np.abs(residual_vector)):.2e}")

print("\nПрогоночные коэффициенты:")
print("  α (первые 5):", np.round(alpha_coeffs[:5], 4))
print("  γ (первые 5):", np.round(gamma_vals[:5], 4))


2. РЕАЛИЗАЦИЯ МЕТОДА ПРОГОНКИ (АЛГОРИТМ ТОМАСА)

Решение системы методом прогонки...

Решение x:
  x[1] = 0.828827
  x[2] = 1.789322
  x[3] = 0.056537
  x[4] = 1.016208
  x[5] = 1.474221
  x[6] = -0.669296
  x[7] = 2.264785
  x[8] = 0.127893
  x[9] = -0.003115
  x[10] = 1.909240

Проверка решения:
  Норма невязки ||Ax - d|| = 1.60e-15
  Максимальная невязка = 8.88e-16

Прогоночные коэффициенты:
  α (первые 5): [-0.25   -0.3779 -0.2629 -0.4044 -0.1622]
  γ (первые 5): [4.     3.8386 2.4955 3.3787 3.2101]


In [5]:
#Сравниваем с методом гаусса
print("\n" + "="*60)
print("3. СРАВНЕНИЕ С МЕТОДОМ ГАУССА")
print("="*60)
def create_tridiagonal_matrix(a, b, c):
    n = len(b)
    A = np.zeros((n, n))
    for i in range(n):
        A[i, i] = b[i]
        if i > 0:
            A[i, i-1] = a[i]
        if i < n-1:
            A[i, i+1] = c[i]
    return A
A_full = create_tridiagonal_matrix(a, b, c)
print("\nа) Решение методом Гаусса")
x_gauss = np.linalg.solve(A_full, d)
print("Решение (первые 5 значений):", np.round(x_gauss[:5], 6))
diff = np.linalg.norm(x_solution - x_gauss)
print(f"\nб) Сравнение решений:")
print(f"  Разница между методами: ||x_прогонка - x_Гаусса|| = {diff:.2e}")


3. СРАВНЕНИЕ С МЕТОДОМ ГАУССА

а) Решение методом Гаусса
Решение (первые 5 значений): [0.828827 1.789322 0.056537 1.016208 1.474221]

б) Сравнение решений:
  Разница между методами: ||x_прогонка - x_Гаусса|| = 2.95e-16


In [6]:
print("\n" + "="*60)
print("4. ОЦЕНКА ЧИСЛА ОПЕРАЦИЙ И СРАВНЕНИЕ")
print("="*60)

def estimate_operations(n):
    thomas_ops = 8 * n
    gauss_ops = (2/3) * n**3 + 2 * n**2
    return thomas_ops, gauss_ops
thomas_ops, gauss_ops = estimate_operations(n)
print(f"\nОценка числа арифметических операций для n = {n}:")
print("\nМетод прогонки (Томаса):")
print(f"  Прямой ход: ~{4*n} умножений/делений + ~{3*n} сложений/вычитаний")
print(f"  Обратный ход: ~{n} умножений + ~{n} сложений")
print(f"  ВСЕГО: ~{thomas_ops:.0f} операций")
print("\nМетод Гаусса:")
print(f"  LU-разложение: ~{(2/3)*n**3:.0f} операций")
print(f"  Решение треугольных систем: ~{2*n**2:.0f} операций")
print(f"  ВСЕГО: ~{gauss_ops:.0f} операций")
ratio = gauss_ops / thomas_ops
print(f"\nСРАВНЕНИЕ ЭФФЕКТИВНОСТИ:")
print(f"Метод Гаусса требует в {ratio:.1f} раз больше операций")
print(f"чем метод прогонки для трехдиагональных систем")
print("\nСравнение для разных размеров матриц:")
print("n\tПрогонка\tГаусс\t\tОтношение (Гаусс/Прогонка)")
print("-" * 60)
for n_test in [10, 50, 100, 500, 1000]:
    ops_thomas, ops_gauss = estimate_operations(n_test)
    ratio_test = ops_gauss / ops_thomas
    print(f"{n_test}\t{ops_thomas:8.0f}\t{ops_gauss:12.0f}\t{ratio_test:10.1f}")


4. ОЦЕНКА ЧИСЛА ОПЕРАЦИЙ И СРАВНЕНИЕ

Оценка числа арифметических операций для n = 10:

Метод прогонки (Томаса):
  Прямой ход: ~40 умножений/делений + ~30 сложений/вычитаний
  Обратный ход: ~10 умножений + ~10 сложений
  ВСЕГО: ~80 операций

Метод Гаусса:
  LU-разложение: ~667 операций
  Решение треугольных систем: ~200 операций
  ВСЕГО: ~867 операций

СРАВНЕНИЕ ЭФФЕКТИВНОСТИ:
Метод Гаусса требует в 10.8 раз больше операций
чем метод прогонки для трехдиагональных систем

Сравнение для разных размеров матриц:
n	Прогонка	Гаусс		Отношение (Гаусс/Прогонка)
------------------------------------------------------------
10	      80	         867	      10.8
50	     400	       88333	     220.8
100	     800	      686667	     858.3
500	    4000	    83833333	   20958.3
1000	    8000	   668666667	   83583.3
